In [12]:
import pandas as pd
import numpy as np
import joblib
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Load and filter the dataset
data = pd.read_excel('final.xlsx')

# Filter only non-renewable energy plants (Gas, Nuclear, Thermal)
non_renewable_data = data[data['Type Of Station'].isin(['Gas', 'Nuclear', 'Thermal'])].copy()

# Ensure 'Date' column is in datetime format
non_renewable_data['Date'] = pd.to_datetime(non_renewable_data['Date'])

# Set the 'Date' column as index
non_renewable_data = non_renewable_data.set_index('Date')

# Function to check if a plant is operational and fill missing values
def preprocess_plant_data(group):
    # Ensure the index is unique before reindexing
    group = group.loc[~group.index.duplicated(keep='first')]
    
    # Fill missing dates with 0 if no data
    group = group.asfreq('D').fillna(0)
    
    # Fill missing values based on whether the plant is operational
    if group['Declared Capability (MWh)'].eq(0).all():
        group['Declared Capability (MWh)'].fillna(0, inplace=True)
    else:
        # Fill missing values with the average of the week (7-day rolling average)
        group['Declared Capability (MWh)'] = group['Declared Capability (MWh)'].fillna(
            group['Declared Capability (MWh)'].rolling(window=7, min_periods=1).mean()
        )
    
    return group

# Apply preprocessing to each plant
non_renewable_data = non_renewable_data.groupby('Station').apply(preprocess_plant_data)

# Drop rows with missing values in other important columns
non_renewable_data.dropna(subset=['Declared Capability (MWh)'], inplace=True)

# Aggregate data at the daily level
daily_data = non_renewable_data.groupby('Date')['Declared Capability (MWh)'].sum()

# Train-test split
train_size = int(len(daily_data) * 0.8)
train, test = daily_data[:train_size], daily_data[train_size:]

# Build and train the ARIMA model
model = ARIMA(train, order=(5,1,0))  # Example order; adjust based on model selection criteria
model_fit = model.fit()  # Removed disp=0

# Make predictions
predictions = model_fit.forecast(steps=len(test))

# Calculate the error
error = mean_squared_error(test, predictions)
print(f'Mean Squared Error: {error}')

# Save the trained model
joblib.dump(model_fit, 'energy_consumption_arima_model.joblib')
print("Model training complete and files saved.")


C:\Users\suyas\AppData\Local\Temp\ipykernel_17064\1441571564.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group = group.asfreq('D').fillna(0)
C:\Users\suyas\AppData\Local\Temp\ipykernel_17064\1441571564.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group = group.asfreq('D').fillna(0)
C:\Users\suyas\AppData\Local\Temp\ipykernel_17064\1441571564.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the fut

Mean Squared Error: 94290548262.4169
Model training complete and files saved.


C:\Users\suyas\AppData\Local\Temp\ipykernel_17064\1441571564.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_renewable_data = non_renewable_data.groupby('Station').apply(preprocess_plant_data)
C:\Users\suyas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\suyas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequen